**Note: This is my EDA.  It's not pretty or nice and probably has some mistakes along the way.  Check out the associated `.py` file for a linted/formatted version of the solution.**

In [2]:
import datetime
from pathlib import Path
import sys
import math

import polars as pl

# Gross hack to get jupyter to find hod stuff.
sys.path.append(str(Path(Path.cwd().parent)))
from hod.utils import get_data, get_male_names

# The Bargain Hunter

(JAMES: Emphasis below mine when I think something will be useful.)

REF: https://hanukkah.bluebird.sh/5784/6/

“Why yes, I did have that rug for a little while in my living room! My cats can’t see a thing but they sure chased after the squirrel on it like it was dancing in front of their noses.

“It was a nice rug and they were surely going to ruin it, so I gave it to my cousin, who was moving into a new place that had wood floors.

“She refused to buy a new rug for herself–she said they were way too expensive. She’s always been very frugal, and she clips every coupon and shops every sale at Noah’s Market. In fact I like to tease her that Noah actually loses money whenever she comes in the store.

“I think she’s been taking it too far lately though. Once the subway fare increased, she stopped coming to visit me. And she’s really slow to respond to my texts. I hope she remembers to invite me to the family reunion next year.”

Can you find her cousin’s phone number?

In [3]:
data = get_data(data_path=Path('./../data/5784/'))

df_customers = data["customers"]
df_orders = data["orders"]
df_orders_items = data["orders_items"]
df_products = data["products"]

df_sale_items = df_orders_items.join(df_products, on="sku").filter(pl.col("unit_price") < pl.col("wholesale_cost")).select(["orderid", "qty", "unit_price", "wholesale_cost"])

df_sale_customers = (
    df_sale_items
    .join(df_orders, on="orderid")
    .join(df_customers, on="customerid")
    .drop(["ordered", "shipped", "address", "birthdate", "timezone", "lat", "long"])
)

df_grouped_sale_customers = (
    df_sale_customers
    .group_by(["customerid", "name", "citystatezip", "phone"])
    .agg(
        [
            pl.col("unit_price").sum().alias("total_unit_price"),
            pl.col("wholesale_cost").sum().alias("total_wholesale_cost") 
        ]
    )
)

df_grouped_sale_customers_sorted = df_grouped_sale_customers.with_columns((pl.col("total_wholesale_cost") - pl.col("total_unit_price")).alias("store_lost_dollars")).sort("store_lost_dollars", descending=True)


df_customers.filter(pl.col("customerid").is_in(df_grouped_sale_customers_sorted.head(1).get_column("customerid"))).drop(["ordered", "shipped", "address", "birthdate", "timezone", "lat", "long"])

customerid,name,citystatezip,phone
i64,str,str,str
4167,"""Sherri Long""","""Bronx, NY 1047…","""585-838-9161"""
